In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd diffusers

In [ ]:
!pip install -e .

In [ ]:
!pip install -r examples/dreambooth/requirements.txt

In [ ]:
!pip install bitsandbytes

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key = "wandb_api_key") # replace with your wandb key

In [ ]:
!accelerate launch examples/dreambooth/train_dreambooth.py \
    --pretrained_model_name_or_path="SG161222/Realistic_Vision_V6.0_B1_noVAE" \
    --instance_data_dir="image_data_dir" \
    --class_data_dir="/kaggle/working/class_images" \
    --output_dir="/kaggle/working/dreambooth_output" \
    --with_prior_preservation \
    --prior_loss_weight=1.0 \
    --instance_prompt="a photo of <irit> woman" \
    --class_prompt="a photo of a woman in plain clothes" \
    --resolution=512 \
    --train_batch_size=1 \
    --use_8bit_adam \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --gradient_accumulation_steps=1 \
    --learning_rate=5e-6 \
    --num_class_images=50 \
    --max_train_steps=1000 \
    --report_to="wandb" \
    --logging_dir="logs" \
    --seed="0"

In [ ]:
%cd /kaggle/working
!rm -rf diffusers

In [11]:
import sys

# Xoá module diffusers khỏi cache
if "diffusers" in sys.modules:
    del sys.modules["diffusers"]

# Loại bỏ tất cả đường dẫn trỏ đến local diffusers
sys.path = [p for p in sys.path if "diffusers" not in p]

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git --quiet

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import os
from safetensors.torch import load_file
import numpy as np 
model_id = "/kaggle/working/dreambooth_output"
path_id = "SG161222/Realistic_Vision_V6.0_B1_noVAE"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
# state_dict = load_file("/kaggle/working/dreambooth_output/checkpoint-1500/unet/diffusion_pytorch_model.safetensors")

# pipe.unet.load_state_dict(state_dict)
prompt = "a professional photo portrait of <irit> woman holding flowers, model photoshoot, professional photo, white background, Amazing Details, Best Quality, Masterpiece, dramatic lighting highly detailed, analog photo, overglaze, 80mm Sigma f/1.4 or any ZEISS lens"
negative_prompt = "asian,multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)
for i in range(10):
    seed = np.random.randint(0, 999999)
    generator = torch.manual_seed(seed)

    image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=50, guidance_scale=8, generator=generator).images[0]
    image.save(f"{output_dir}/image_{i+1:02d}_seed{seed}.png")